In [1]:
import numpy as np
import pandas as pd
import math


#PARAMETERS
T= 15  #Number of iterations for a given run
N = 5 #Number of runs
DEBUG_PRINT = 1

#Use this to implement the "framework"


#User libraries
#from filename import class
from mab_base import mab_base
from collab_filter import MF
import thompson_algo as ta
import UCB as ucb
import Epsilon_Greedy as ep


#Python libaries
from math import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

#def DCG_compute():
#   
##   Implementing equation 3    
#def DCG_Normalize(self,n_users,dcg_true,dcg_pred):
#	for index in range(len(dcg_true)) :
#		ndcg += dcg_pred[index]/dcg_user[index]
#	return ndcg/n_users
#    
##   Implementing equation 4


def DCG_Normalize(dcg_pred,dcg_true):
    assert(len(dcg_true) == len(dcg_pred) )
    assert(len(dcg_true) > 0)
    ndcg = 0
    for index in range(len(dcg_true)) :
        if(dcg_true[index] != 0):
            ndcg += dcg_pred[index]/dcg_true[index]
        else:
            assert(1)
    print(dcg_true)
    return ndcg/len(dcg_true)


def DCG_Pred_perUser(r_u_t):
    dcg = r_u_t[0]
    for time in range(len(r_u_t)) :
        if (time > 1) :
            dcg += r_u_t[time]/(math.log(time,2)) 
    return dcg

def printClusters(mab):
    print('Current Cluster State:')
    print(mab.clusters)
    print('')
    print('Original/Safe Clusters:')
    print(mab.clusters_safe)

def thompsonState(mab):
    print('--- MAB Status -------------------------------------------')
    print('Arms:\t\t', end = '')
    print(mab.k)
    print('')
    print('Iterations:\t', end='')
    print(mab.n)
    print('')
    print('Arm Alpha:\t', end='')
    print(mab.k_a)
    print('')
    print('Arm Beta:\t', end='')
    print(mab.k_b)
    print('')
    print('Arm Theta:\t', end='')
    print(mab.k_theta)
    print('')
    print('Arm Reward:\t', end='')
    print(mab.k_reward)
    print('')
    printClusters(mab)
    print('----------------------------------------------------------')
    
def getRewards(X_test, rec_movie):  #get true value of current user and the recommended movie/or recommended cluster
    temp = X_test.iloc[:,rec_movie:rec_movie+1]
    reward = temp[rec_movie].values[0]/np.max(X_test)[0]    
    return reward

#STEP1: Read csvs and create nice DS out of them
	
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

# Merge the two files
ratings = pd.merge(movies,ratings,on='movieId')

# Creating the sparse matrix
userRatings = ratings.pivot_table(index=['userId'],columns=['movieId'],values='rating').fillna(0)
userRatings.head()

# Creating predicted rating matrix
R = userRatings.values

#Creating the Collab filtering object
mf = MF(R, K=2, alpha=0.1, beta=0.01, iterations=1)

training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()
print("Global bias:")
print(mf.b)
print()
print("User bias:")
print(mf.b_u)
print()
print("Item bias:")
print(mf.b_i)
print("MSE:")
print(mf.mse())

final_predicted_matrix = pd.DataFrame(mf.full_matrix())
print(final_predicted_matrix)



P x Q:
[[4.89778525 4.51745843 4.59730187 ... 4.3757311  4.49273284 4.56029972]
 [4.1000292  3.3898996  3.19322334 ... 4.30339555 3.30909025 3.82531351]
 [3.11264239 2.83668378 2.87819494 ... 2.38910339 2.77225351 2.7572999 ]
 ...
 [3.47957807 2.76537871 3.4695925  ... 3.49011376 3.09674425 3.19121455]
 [3.60837207 3.31721547 3.29560125 ... 2.92950416 3.22719805 3.25672569]
 [4.00035867 3.54003211 3.45895779 ... 3.67520833 3.45885763 3.67959579]]

Global bias:
3.501556983616962

User bias:
[ 9.70011949e-01  2.09228346e-01 -8.37596640e-01 -1.02451431e-01
 -6.63076446e-02  8.59189305e-02 -5.58117647e-01  1.18285367e-01
 -5.03213241e-01 -4.01447970e-01  2.77884880e-01  9.85853941e-01
  9.36260315e-02 -4.65207692e-02 -5.61212636e-01 -4.09757749e-01
  3.31275678e-01 -1.13375634e-01 -8.86191171e-01  1.03166860e-01
 -2.64457593e-01 -1.85532892e+00 -5.35157572e-02 -1.90044389e-01
  5.69265239e-01 -1.18165255e-01 -3.08382044e-01 -7.03778518e-01
  1.25047037e-01  6.75500409e-01  2.18625019e-01 

277.2377528145752
         0         1         2         3         4         5         6     \
0    4.897785  4.517458  4.597302  3.921676  4.283428  5.094468  4.124394   
1    4.100029  3.389900  3.193223  2.981174  3.654914  3.642430  3.452447   
2    3.112642  2.836684  2.878195  2.178592  2.455594  3.433247  2.265419   
3    3.845936  3.514036  3.456980  2.886394  3.222056  4.021431  3.006085   
4    3.934580  3.616728  2.976187  2.966383  3.354890  3.744710  2.928446   
5    4.008514  3.597556  3.672102  3.019652  3.408116  4.157895  3.252319   
6    3.342352  2.879063  3.099792  2.335545  2.752396  3.513470  2.657499   
7    4.133785  3.860729  3.145651  3.182202  3.540787  3.958733  3.080258   
8    3.496258  3.402233  3.259219  2.632608  2.774953  3.955125  2.488510   
9    3.432108  2.708784  3.152884  2.323678  2.937479  3.378941  2.965779   
10   4.276706  3.926908  3.241877  3.294244  3.714938  4.011764  3.277972   
11   4.861813  4.416724  4.963063  3.870565  4.235720  5.2

In [2]:
dcg_thom=[]
dcg_ucb = []
dcg_greedy = []
dcg_thom_true=[]
dcg_ucb_true = []
dcg_greedy_true = []
dcg_rnn = []
dcg_rnn_true = []

In [3]:
for i in range(0,N):
    #Add cluster classification here
    # Creating Clusters
    # print(i)
    #LOO here 
    X_train, X_test = train_test_split(final_predicted_matrix, test_size=0.001)
    print('')
    user = X_test.index[0]
    print(user)
    print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')


    sse = []
    #list_k = list(range(4, 20))

    #for k in list_k:
    #km = KMeans(n_clusters=5,n_init = 20, n_jobs=-1)
    #km.fit(X_train)
    #sse.append(km.inertia_)

    # Plot sse against k
    #plt.figure(figsize=(6, 6))
    #plt.plot(list_k, sse, '-o')
    #plt.xlabel(r'Number of clusters *k*')
    #plt.ylabel('Sum of squared distance');


    model_knn = KMeans(n_clusters=3, n_init = 20, random_state = 42, n_jobs=-1)
    model_knn.fit(X_train)

    # new cluster labels

    y_km = model_knn.fit_predict(X_train)
    final_clusters = model_knn.cluster_centers_

    # Final Clusters
    final_clusters = pd.DataFrame(final_clusters)

    # Creating a new initial column with cluster name
    new_col = {'cluster': [i for i in range(1,len(final_clusters)+1)]}
    new_col = pd.DataFrame(new_col)
    #Setting cluster for other algorithms to use
    final_clusters = new_col.join(final_clusters)
    # print("Done")

    # create thompson mab object
    r_u_t_thom  = []
    r_u_t_thom_true  = []

    mab_0 = ta.thompson()    
    mab_0.set_clusters(final_clusters)
    #thompsonState(mab_0)
    last_reward = 0
    rnn_array = [0, 0]
    for i in range(0,T):
        
        
        #print("Movie Recommended:\t", end='')
        rec_movie_id = mab_0.pull_arm(last_reward)
        #print(rec_movie_id)
        
        
        #print('')
        last_reward = getRewards(X_test,rec_movie_id) 
        
        temp = X_test.iloc[:,rec_movie_id:rec_movie_id+1]
        
        r_u_t_thom.append(temp[rec_movie_id].values[0])
        r_u_t_thom_true.append(np.max(X_test,axis =1).values[0])
        
        rnn_array[0] = rec_movie_id


    dcg_thom.append(DCG_Pred_perUser(r_u_t_thom))
    
    dcg_thom_true.append(DCG_Pred_perUser(r_u_t_thom_true))
    #print("True DCG for current user for Thompson: ", dcg_thom_true)
    #print("DCG for current user for Thompson: ", dcg_thom)





40
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

576
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

303
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

31
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

33
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

13
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

303
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

417
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

607
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

164
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

442
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

563
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

193
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

322
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

433
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

94
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

342
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

196
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

584
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

195
^^^^^^^^^^^^

In [4]:
ndg_thom = DCG_Normalize(dcg_thom,dcg_thom_true)

print("NDCG for Thompson Algorithm for ",N, "users over time ",T, ": ", ndg_thom)



[25.703372471336813, 31.03923702418966, 34.69888073173061, 29.00237258119478, 42.33646174128061, 30.103151751159682, 34.69888073173061, 31.86029338498328, 37.3081512682868, 35.28119183696301, 37.725574840833346, 31.69345702780206, 30.506160996614092, 32.64874257696944, 29.16258058591368, 31.112469265469077, 31.334763120623585, 30.008415221232205, 36.11101987886186, 30.002003373465705]
NDCG for Thompson Algorithm for  20 users over time  15 :  0.9055612533174079
